アンサンブル用シート

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib

###訓練データの読み込み
house_age = pd.read_csv('../data/handmade_data/house_age.csv')
area_size = pd.read_csv('../data/handmade_data/area_size.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
contract_span = pd.read_csv('../data/handmade_data/contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('../data/handmade_data/contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('../data/handmade_data/reg_rent.csv')
floor_scores = pd.read_csv('../data/handmade_data/floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('../data/handmade_data/floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('../data/handmade_data/capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('../data/handmade_data/capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('../data/handmade_data/stations.csv')
minits = pd.read_csv('../data/handmade_data/minits.csv')
addresses = pd.read_csv('../data/handmade_data/addresses.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
buildings = pd.read_csv('../data/handmade_data/buildings.csv')

rent = pd.read_csv('../data/handmade_data/rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('../data/test_data/test_house_age.csv')
test_area_size = pd.read_csv('../data/test_data/test_area_size.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_contract_span = pd.read_csv('../data/test_data/test_contract_span.csv')
test_contract_span2 = pd.read_csv('../data/test_data/test_contract_span2.csv')
test_reg_rent = pd.read_csv('../data/test_data/test_reg_rent.csv')
test_floor_scores = pd.read_csv('../data/test_data/test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('../data/test_data/test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('../data/test_data/test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('../data/test_data/test_capital_floor_scores2.csv')
test_stations = pd.read_csv('../data/test_data/test_stations.csv')
test_minits = pd.read_csv('../data/test_data/test_minits.csv')
test_addresses = pd.read_csv('../data/test_data/test_addresses.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_buildings = pd.read_csv('../data/test_data/test_buildings.csv')

Duplicate key in file PosixPath('/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc'), line 258 ('font.family : IPAexGothic')
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ensemble_result = []
ensemble_result = pd.DataFrame(ensemble_result)

GBDT_result = pd.read_csv('../data/result_data/GBDT_cv_result.csv')
DART_result = pd.read_csv('../data/result_data/DART_cv_result.csv')
GOSS_result = pd.read_csv('../data/result_data/GOSS_cv_result.csv')
XGB_result = pd.read_csv('../data/result_data/XGB_cv_result.csv')

ensemble_result = (GBDT_result['0'] + DART_result['0']+ GOSS_result['0'] + XGB_result['0'])/4
ensemble_result.reset_index()

,index,0
0,0,130823.362045
1,1,122442.284617
2,2,76058.348332
3,3,60968.183027
4,4,112185.554535
...,...,...
31257,31257,109108.945069
31258,31258,101232.084907
31259,31259,176062.820110
31260,31260,258579.189046


In [5]:
df = pd.read_csv('../data/outer_data/test.csv')
id = df['id']
result = pd.concat([id, ensemble_result],axis=1)
result.to_csv('../data/result_data/result.csv',index=False)

In [12]:
predicts = pd.concat([lightGBM_result,DART_result,GOSS_result],axis=1)
predicts.corr()

,id,0,id,0,id,0
id,1.000000,0.005236,1.000000,0.003636,1.000000,0.004711
0,0.005236,1.000000,0.005236,0.990025,0.005236,0.990926
id,1.000000,0.005236,1.000000,0.003636,1.000000,0.004711
0,0.003636,0.990025,0.003636,1.000000,0.003636,0.986821
id,1.000000,0.005236,1.000000,0.003636,1.000000,0.004711
0,0.004711,0.990926,0.004711,0.986821,0.004711,1.000000
